In [2]:
import numpy as np
import pandas as pd

# Validating CI on Test Set

In [3]:
data = pd.read_csv("genetech.batch_norm_genes.tsv", sep="\t")
clinical = pd.read_csv("IMvigor210.clinical", sep="\t")
model_path = "/Users/Peter/Documents/GitHub/risk/models/all_immune_genes/checkpoint.0220.hdf5"

IOError: File genetech.batch_norm_genes.tsv does not exist

In [31]:
from keras.models import load_model
from keras import backend as K
import tensorflow as tf

def negative_log_partial_likelihood(censor, risk):
    """Return the negative log-partial likelihood of the prediction
    y_true contains the survival time
    risk is the risk output from the neural network
    censor is the vector of inputs that are censored
    regularization is the regularization constant (not used currently in model)

    Uses the Keras backend to perform calculations

    Sorts the surv_time by sorted reverse time
    """

    # calculate negative log likelihood from estimated risk
    epsilon = 0.001
    hazard_ratio = K.exp(risk)

    # cumsum on sorted surv time accounts for concordance
    log_risk = K.log(tf.cumsum(hazard_ratio+epsilon))
    uncensored_likelihood = risk - log_risk

    # apply censor mask: 1 - dead, 0 - censor
    censored_likelihood = uncensored_likelihood * censor
    num_observed_events = K.sum(censor)
    neg_likelihood = - K.sum(censored_likelihood) / \
        tf.cast(num_observed_events, tf.float32)
    return neg_likelihood

In [32]:
surv_model = load_model(model_path, custom_objects={
                                        'negative_log_partial_likelihood': negative_log_partial_likelihood})
surv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              1076224   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
__________

In [33]:
from lifelines.utils import concordance_index

def concordance_metric(survival_time, predicted_risk, censor):
    # calculate the concordance index
    epsilon = 0.001
    partial_hazard = np.exp(-(predicted_risk+epsilon)).flatten()
    censor = censor.astype(int)
    ci = concordance_index(survival_time, partial_hazard, censor)
    return ci

predictions = surv_model.predict(data)
concordance_metric(clinical["OS"], predictions, clinical['Event'])

0.5247751719273497

# Transfer Learning

## Simple Cox Regression

In [6]:
labels = clinical[['Response']].values
index = ~np.isnan(labels).flatten()
labels = labels[index,]
data = data.values
data = data[index,:]

# split the data into a training set and a validation set
VALIDATION_SPLIT = 0.8

# indices = np.arange(tpm.shape[0])
# np.random.shuffle(indices)
# # tpm = tpm[indices]
# labels = surv_time[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:num_validation_samples]
y_train = labels[:num_validation_samples]
x_val = data[num_validation_samples:]
y_val = labels[num_validation_samples:]

In [7]:
from keras.models import Sequential
from keras.models import Model

surv_model = Sequential()
surv_model = load_model(model_path, custom_objects={
                                        'negative_log_partial_likelihood': negative_log_partial_likelihood})

# taking out the last layer layers
inp = surv_model.input
surv_model.pop()
surv_model.pop()
surv_model.pop()

surv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               269056    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
__________

/anaconda3/envs/surv/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [8]:
from keras.optimizers import Adam

# to test with just linear cox regression
cox_model = surv_model

# freeze all the layers to test the weights
for layer in cox_model.layers:
    layer.trainable = False
    
opt = Adam(lr=3e-2)

loss_fn = negative_log_partial_likelihood
cox_model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

cox_output = cox_model.predict(data)
cox_output.shape

(298, 128)

## Transfer Learning with Deep Net

In [1]:
from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE

data = pd.read_csv("genetech.batch_norm_genes.tsv", sep="\t")
clinical = pd.read_csv("IMvigor210.clinical", sep="\t")

labels = clinical[['Response']].values
index = ~np.isnan(labels).flatten()
labels = labels[index,]
data = data.values
data = data[index,:]

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.20, stratify=labels.flatten())

# sm = SMOTE()
# x_train_res, y_train_res = sm.fit_sample(x_train, y_train.ravel())
# x_val_res, y_val_res = sm.fit_sample(x_val, y_val.ravel())
# y_train_res_cat = np.append(y_train_res.reshape([-1,1]), np.abs(y_train_res - 1).reshape([-1,1]), axis=1)
# y_val_res_cat = np.append(y_val_res.reshape([-1,1]), np.abs(y_val_res - 1).reshape([-1,1]), axis=1)

NameError: name 'pd' is not defined

In [52]:
from keras.models import Sequential
from keras.models import Model

surv_model = Sequential()
surv_model = load_model(model_path, custom_objects={
                                        'negative_log_partial_likelihood': negative_log_partial_likelihood})

# taking out the last layer layers
surv_model.pop()
xfer_model = surv_model

xfer_model.add(layers.Dense(1, activation="sigmoid", name="xfer_output"))

# freeze all the layers before the output layer
for layer in xfer_model.layers[0:-4]:
    layer.trainable = False

xfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              1076224   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
__________

/anaconda3/envs/surv/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [54]:
from keras.optimizers import Adam

opt = Adam(lr=3e-2)

xfer_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

xfer_model.fit(x_train_res, y_train_res,
          batch_size=64,
          epochs=30,
          validation_data=(x_val_res, y_val_res))

Train on 368 samples, validate on 92 samples
Epoch 1/30
368/368 [==============================] - 10s 28ms/step - loss: 6031576.5870 - acc: 0.4918 - val_loss: 4271704.9783 - val_acc: 0.4783
Epoch 2/30
368/368 [==============================] - 1s 4ms/step - loss: 6061449.4457 - acc: 0.5136 - val_loss: 4271705.0163 - val_acc: 0.4891
Epoch 3/30
368/368 [==============================] - 1s 4ms/step - loss: 6015499.7609 - acc: 0.4728 - val_loss: 4271704.9783 - val_acc: 0.4674
Epoch 4/30
368/368 [==============================] - 2s 4ms/step - loss: 6089169.9674 - acc: 0.5326 - val_loss: 4271705.0163 - val_acc: 0.5000
Epoch 5/30
368/368 [==============================] - 1s 4ms/step - loss: 6002937.6957 - acc: 0.5136 - val_loss: 4271704.9783 - val_acc: 0.5000
Epoch 6/30
368/368 [==============================] - 1s 4ms/step - loss: 6053548.4674 - acc: 0.4918 - val_loss: 4271704.9783 - val_acc: 0.4022
Epoch 7/30
368/368 [==============================] - 1s 4ms/step - loss: 6074454.1848 - 

KeyboardInterrupt: 

In [45]:
xfer_model.predict(x_train)

array([[0.20991541],
       [0.21272932],
       [0.21062167],
       [0.21154906],
       [0.21007502],
       [0.20977132],
       [0.21046393],
       [0.21228957],
       [0.21004291],
       [0.21030395],
       [0.20979358],
       [0.20966385],
       [0.2102215 ],
       [0.21104631],
       [0.21001796],
       [0.21006668],
       [0.21272932],
       [0.21000938],
       [0.2108271 ],
       [0.21001098],
       [0.20981066],
       [0.20979513],
       [0.21161972],
       [0.20965242],
       [0.21039522],
       [0.21272932],
       [0.20981507],
       [0.21048294],
       [0.21017742],
       [0.21062434],
       [0.2100706 ],
       [0.20941243],
       [0.20964184],
       [0.20964791],
       [0.2104498 ],
       [0.21036242],
       [0.2094543 ],
       [0.2096082 ],
       [0.20993097],
       [0.2098705 ],
       [0.21017985],
       [0.21002088],
       [0.20998134],
       [0.20961434],
       [0.20950387],
       [0.21272932],
       [0.20996553],
       [0.209

## Predicting on Top Genes from Linear Regression

In [12]:
from sklearn.model_selection import train_test_split

data = pd.read_csv("tpm.xfer.X.txt", sep="\t")
labels = pd.read_csv("tpm.xfer.y.txt", sep="\t")

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.20)

In [15]:
from keras.models import Sequential
from keras.models import Model
from keras import layers

surv_model = Sequential()
surv_model = load_model(model_path, custom_objects={
                                        'negative_log_partial_likelihood': negative_log_partial_likelihood})

# taking out the last layer layers
surv_model.pop()
surv_model.pop()
surv_model.pop()

reg_model = surv_model
reg_model.add(layers.Dense(64, activation='relu', name="xfer_dense1"))
reg_model.add(layers.BatchNormalization(name="xfer_batchnorm_1"))
reg_model.add(layers.Dense(14, activation="relu", name="output_dense"))

# freeze all the layers before the output layer
for layer in reg_model.layers[0:-3]:
    layer.trainable = False

reg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               269056    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
__________

/anaconda3/envs/surv/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [17]:
opt = Adam(lr=0.03)

reg_model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

reg_model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          validation_data=(x_val, y_val))

ValueError: Error when checking input: expected dense_1_input to have shape (1050,) but got array with shape (945,)

### Predicting ICB response based on these outputs

In [ ]:
reg_model.add(layers.Dense(1, activation="sigmoid", name="reg_output"))

# freeze all the layers before the output layer
for layer in reg_model.layers[0:-2]:
    layer.trainable = False

opt = Adam(lr=0.03)

reg_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
data = pd.read_csv("genetech.batch_norm_genes.tsv", sep="\t")
clinical = pd.read_csv("IMvigor210.clinical", sep="\t")

labels = clinical[['Response']].values
index = ~np.isnan(labels).flatten()
labels = labels[index,]
data = data.values
data = data[index,:]

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.33, stratify=labels.flatten())

In [ ]:
reg_model.fit(x_train, y_train,
          batch_size=64,
          epochs=20,
          validation_data=(x_val, y_val))

# tSNE Viz

In [ ]:
# taking out the last 2 layers
from keras.models import Model
inp = surv_model.input
output = surv_model.layers[-3].output
model = Model(inp, output)
model.summary()

In [ ]:
data = pd.read_csv("genetech.batch_norm_genes.tsv", sep="\t")

results = model.predict(data)
idx = ~np.isnan(clinical["Response"])
# clinical = clinical.values
clinical = clinical.loc[idx,]
results = results[idx,]

In [ ]:
import time

from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=1, perplexity=50, n_iter=10000)
tsne_results = tsne.fit_transform(results)

In [ ]:
clinical_feature = clinical['Response']

target_ids = range(int(min(clinical_feature)), int(max(clinical_feature))+1)

from matplotlib import pyplot as plt
plt.figure(figsize=(6, 5))
colors = 'r', 'g', 'b', 'c', 'm', 'y', 'k', 'w', 'orange', 'purple'
for i, c in zip(target_ids, colors):
    plt.scatter(tsne_results[clinical_feature == i, 0], tsne_results[clinical_feature == i, 1], c=c)
# plt.legend()
plt.show()

In [ ]:
clinical_feature = clinical['Best']

target_ids = range(int(min(clinical_feature)), int(max(clinical_feature))+1)